In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, 'adcc95ed-0e6f-494f-a3a1-d5e74e038455', 'p-0e7c11d93da5347766da7d0e0924866399a71d41')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200616165020-0000
KERNEL_ID = 38f05806-533a-4ddf-ade0-c64463c31cf3


In [16]:
import ibmos2spark
import nltk
# !pip install gensim 
from gensim.parsing.preprocessing import STOPWORDS

In [3]:
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-283f9a60-2385-40f6-973d-6cd059b4731f',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '8-VH-6DHOLwKlgloUuzelkhXgt0elMaOQxDZB1EznoCu'
}

configuration_name = 'os_c42173278f6a4c65aaac64664b8be689_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('data.csv', 'bdaproject-donotdelete-pr-qh7hn4gnoctssn'))

In [4]:
import nltk
alphabets = []
tokenizer = nltk.RegexpTokenizer(r"\w+")
for i in range(97,123):
    alphabets.append(chr(i))
def cleaning(line):
    rem_garbage = line.replace("NEWLINE_TOKEN",'').replace("_"," ").replace("1"," ").replace("2"," ").replace("3"," ").replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ").replace("8"," ").replace("9"," ").replace("0"," ").lower()
    rem_punct = tokenizer.tokenize(rem_garbage)
    rem_stopwords = [word for word in rem_punct if not word in STOPWORDS.union(set(alphabets)) or not word.isalpha()]
    return rem_stopwords

In [5]:
def create_pair(word):
    return (word,1)

In [6]:
coments = df_data_1.select("comment").rdd.collect()

In [7]:
clean_coments = []
for i in range(len(coments)):
    clean_coments.extend(cleaning(coments[i][0]))

In [8]:
cleaned_comments = sc.parallelize(clean_coments)
paired_words = cleaned_comments.map(create_pair)

In [9]:
def sum_counts(a, b):
    return a + b

In [10]:
corpus_word_count = paired_words.reduceByKey(sum_counts).collect()

In [11]:
new_df = sqlContext.createDataFrame(corpus_word_count,["Words","Count"]).toPandas()

In [17]:
# !pip install textblob
sentiment = []
from textblob import TextBlob
for i in range(len(new_df)):
    blob = TextBlob(new_df["Words"][i])
    sentiment.append((new_df["Words"][i],new_df["Count"][i],blob.sentiment.polarity))

In [13]:
import pandas as pd
DF = pd.DataFrame(sentiment,columns=["Words","Count","Sentiment_Score"])

In [14]:
WCS_DF = DF[DF["Count"]>20].sort_values(by="Count",ascending = False)
WCS_DF.index = range(len(WCS_DF["Words"]))

In [15]:
project.save_data("Corpus_Word_Count.csv", WCS_DF.to_csv(index=False),overwrite=True)

{'file_name': 'Corpus_Word_Count.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'bdaproject-donotdelete-pr-qh7hn4gnoctssn',
 'asset_id': '4b650cbc-d254-4cd3-8fa4-e27b09d99819'}